In [1]:
from config.configuration import RunDetails

runDetail = RunDetails('config.yml', 'GitLab-training-data')
# runDetail = RunDetails('config.yml', 'ASVspoof-2019_training')
# runDetail = RunDetails('config.yml', 'ASVspoof-2019_training_epoch-100')

notebookName = 'audio-deepfake-detection-training'
plot_title_suffix = "(Training)"

In [2]:
configFilename = runDetail.configFilename
runJobId = runDetail.jobId

In [3]:
import config.configuration as configuration
import model_definitions.model_cnn_definition as model_cnn_definition
from postprocessors.plot_confusion_matrix import PlotConfusionMatrix
from postprocessors.plot_roc_curve import PlotRocCurve
from preprocessors.abstract_preprocessor import AbstractPreprocessor
from preprocessors.preprocessor_factory import PreprocessorFactory
from notebook_utils import notebookToPython
from processors.basic_model_training_processor import BasicModelTrainingProcessor
from processors.basic_model_evaluation_processor import BasicModelEvaluationProcessor

In [4]:
config = configuration.ConfigLoader(configFilename)

notebookToPython(notebookName)
job = config.getJobConfig(runJobId)

if (job.newModelGenerated == False):
    raise ValueError("This notebook is meant for training. Select a job without a value for 'persisted-model' set.")

Write python file
Generating new model name: output/GitLab-training-data_2025-04-17T12-57-44.517459.libjob
Assigned model name: output/GitLab-training-data_2025-04-17T12-57-44.517459.libjob


In [5]:
preproc_factory = PreprocessorFactory()
preprocessor: AbstractPreprocessor = preproc_factory.newPreprocessor(job.preprocessor)

MelSpectrogramPreprocessor


In [6]:
X, y_encoded = preprocessor.extract_features_multipleSource(job, job.dataPathSuffix)

Loading ./testvalues/sksmta.train.trn.txt...
fullDataPath: ./testaudio-sksmta
Loading audio files: 1
Loading audio files: 2
Loading audio files: 3
Loading audio files: 4
Loading audio files: 5
Loading audio files: 6
Loading audio files: 7
Loading audio files: 8
Loading audio files: 9
Loading audio files: 10
Number of audio files load: 10


In [7]:
import joblib
import json
import pytz
from datetime import datetime
from sklearn.model_selection import cross_val_score, train_test_split
from tensorflow.keras.models import Model

from config.configuration import Job
from model_definitions.model_abstract_definition import ModelAbstractDefinition
from processors.abstract_model_training_processor import AbstractModelTrainingProcessor

# =============================================================================
class BasicModelTrainingProcessor2(AbstractModelTrainingProcessor):

    # -------------------------------------------------------------------------
    def __init__(self, job: Job, modelDefType):
        super().__init__(job)
        self.resetStatistics()
        self.modelDefType = modelDefType

    # -------------------------------------------------------------------------
    def resetStatistics(self):
        self.jobStartTime = None
        self.inputFileBatchCount = 0
        self.inputFileCount = 0

    # -------------------------------------------------------------------------
    def process(self, X, y_encoded, channels, test_size = 0.2, trainingSplitRandomState: int = None, 
                        scoring = 'accuracy'):
        
        if (self.jobStartTime == None):
            self.jobStartTime = datetime.now(pytz.utc)

        useTrainingSplitRandomState: int = self.__get_training_split_random_state__(trainingSplitRandomState)

        print(f"Selecting training and test data - traininSplitRandomState: {useTrainingSplitRandomState}")
        X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=test_size, random_state=useTrainingSplitRandomState)
        
        print(f"Training using {len(X_train)} files.")
        model = self.__train_model__(X_train, X_test, y_train, y_test, channels, scoring)

        return model, X_train, X_test, y_train, y_test
    
    # -------------------------------------------------------------------------
    def reportSnapshot(self):
        timestamp_utc = datetime.now(pytz.utc)
        elapsed_time = timestamp_utc - self.jobStartTime
        prettyJson = json.dumps(self.__job__.__dict__, indent=4)

        report = f"---- Training (start) ----\n"
        report = report + f"start time: {self.jobStartTime.isoformat()}\n"
        report = report + f"end time: {timestamp_utc.isoformat()}\n"
        report = report + f"elapsed: {elapsed_time}\n\n"
        report = report + f"model file: {self.__job__.persistedModel}\n"
        report = report + f"batch count: {self.inputFileBatchCount}\n"
        report = report + f"file count: {self.inputFileCount}\n"
        report = report + f"job: {prettyJson}\n\n"
        report = report + f"---- Training (end) ----\n"

        return report

    # -------------------------------------------------------------------------
    def __train_model__(self, X_train, X_test, y_train, y_test, channels, scoring) -> Model:
        
        modelDef: ModelAbstractDefinition = self.modelDefType(self.__job__, X_train.shape[2], channels)

        model = modelDef.buildModel()
        model.compile(optimizer=self.__job__.optimizer, loss=self.__job__.loss, metrics=self.__job__.metrics)

        print("Training the Model...")
        model.fit(X_train, y_train, batch_size=self.__job__.batchSize, epochs=self.__job__.numEpochs, validation_data=(X_test, y_test))

        print(f"Saving model: {self.__job__.persistedModel}")
        joblib.dump(model, self.__job__.persistedModel)

        # scores = cross_val_score(model, X_train, y_train, cv=self.__job__.cv, scoring=scoring)
        # print(f"cross validation scores: {scores}")

        return model

In [8]:
trainingProc = BasicModelTrainingProcessor2(job, model_cnn_definition.ModelCnnDefinition)
model, X_train, X_test, y_train, y_test = trainingProc.process(X, y_encoded, 1)

Selecting training and test data - traininSplitRandomState: 3
Training using 8 files.
__job__: <config.configuration.Job object at 0x00000245FACE3BC0>
Training the Model...
Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.6250 - loss: 14.8835 - val_accuracy: 0.5000 - val_loss: 35.4379
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.3750 - loss: 92.3701 - val_accuracy: 0.5000 - val_loss: 197.0585
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.6250 - loss: 20.9505 - val_accuracy: 0.5000 - val_loss: 207.0571
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.7500 - loss: 164.2165 - val_accuracy: 0.5000 - val_loss: 87.4394
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.6250 - loss: 62.6134 - val_accuracy: 0.5000 - val_loss: 14.3263
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.3750 - loss: 81.8597 - val_accuracy: 1.0000 - val_loss: 7.9274e-06
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 

In [9]:
from scikeras.wrappers import KerasClassifier

kerasModel = KerasClassifier(build_fn=model, batch_size=job.batchSize, epochs=job.numEpochs,
                             optimizer=job.optimizer, loss=job.loss, metrics=job.metrics)

scores = cross_val_score(kerasModel, X_train, y_train, cv=job.cv)
print(f"cross validation scores: {scores}")


AttributeError: 'super' object has no attribute '__sklearn_tags__'

### Test Model

In [ ]:
evaluationProc = BasicModelEvaluationProcessor(job, model)
results = evaluationProc.process(X_test, y_test)
print(f"{results.reportSnaphot()}")


In [ ]:
CM_TITLE = f"{PlotConfusionMatrix.DEFAULT_TITLE} {plot_title_suffix}"
cm_plot = PlotConfusionMatrix()
cm_plot.plotFromResults(results, job, CM_TITLE)


In [ ]:
RC_TITLE = f"{PlotRocCurve.DEFAULT_TITLE} {plot_title_suffix}"
roc_plot = PlotRocCurve()
roc_plot.plotFromResults(results, RC_TITLE)

In [ ]:
print("\n")
report = evaluationProc.reportSnapshot(trainingProc)
evaluationProc.writeReportToFile(job.persistedModelResults, report)

print(report)